<a href="https://colab.research.google.com/github/litongzheng/APS360Project/blob/master/aps360.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import csv
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchtext
import numpy as np
import matplotlib.pyplot as plt

In [0]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130


In [0]:
torch.cuda.get_device_name(0)

'Tesla K80'

In [0]:
# sp = pd.read_csv("All-seasons.csv")
# sp.replace('\n',' ',regex = True,inplace=True)
# sp.to_csv('sp.csv',index= False)
# sp.head()
# cartman = sp[sp.Character=="Cartman"]
# stan = sp[sp.Character == "Stan"]

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
sp_text = ""

chars = []
i = 0
for char in open('/content/drive/My Drive/Colab Notebooks/character.txt'):
  chars.append(char.strip() + ":\t")
  if i>=4000:
    break
  i = i+1
lines = [] 

i = 0
for line in open('/content/drive/My Drive/Colab Notebooks/Line.txt',encoding="ISO-8859-1"):
  lines.append(line)
  if i>=4000:
    break
  i = i+1
        
for i in range(len(lines)):
  total = chars[i] + lines[i]
  sp_text += total


In [7]:
len(sp_text)

292275

In [8]:
vocab = list(set(sp_text))
vocab_stoi = {s: i for i, s in enumerate(vocab)}
vocab_itos = {i: s for i, s in enumerate(vocab)}
len(vocab)

90

In [9]:
import random
random.seed(10)

sp_len = len(sp_text)

def random_chunk(chunk_len=300):
    """Return a random subsequence from `spam_text`"""
    start_index = random.randint(0, sp_len - chunk_len)
    end_index = start_index + chunk_len + 1
    return sp_text[start_index:end_index]

print(random_chunk())

eah. It's a hybrid. I just... I just couldn't sit back and-  be a part of destroying the earth anymore. 
Richard:	Well... Good for you. 
Gerald:	Oho...  Thanks.  
Stephen:	Well, there goes the new high-and-mighty Gerald Broflovski. 
Mrs. Garrison:	Yeah, ever since he got that new hybrid he thinks he'


In [10]:
def text_to_tensor(text, vocab=vocab):
    """Return a tensor containing the indices of characters in `text`."""
    indices = [vocab_stoi[ch] for ch in text]
    return torch.tensor(indices)

print(text_to_tensor(random_chunk()))


tensor([61, 86, 59, 66, 33, 11,  5, 61, 70, 11,  4, 24, 40, 33, 35, 70, 38, 59,
        35,  4, 37, 59, 33, 35, 70, 38, 59, 35,  4, 37, 59, 66, 60, 35, 27, 53,
        38, 24, 40, 56, 61, 35,  5, 38, 59, 27, 14, 59,  4, 35, 47, 86, 86, 86,
        59, 35, 36, 38,  4, 86, 59, 59, 52, 38, 78, 53, 35, 70, 38, 59, 66, 33,
        11,  5, 61, 70, 11,  4, 24, 40, 12, 23, 87,  1, 59, 83, 27,  4, 11, 78,
        78, 16, 37, 59, 66, 33, 78, 38,  5, 58, 24, 40, 52, 38, 78, 53, 35, 70,
        38, 59, 61, 35, 59, 54, 38, 38,  2, 35, 34,  3, 14, 59, 50, 35, 16, 14,
        59, 66, 33, 11,  5, 61, 70, 11,  4, 24, 40,  0, 59, 47, 11,  4, 61, 59,
        35,  4, 38, 59, 35, 83, 59, 61, 87, 35, 14, 38, 59, 61, 27, 70, 38, 77,
        36, 87, 35,  4, 38, 59, 61, 87, 27,  4, 23, 14, 86, 59, 66, 33, 78, 38,
         5, 58, 24, 40, 21, 87, 87,  1, 59, 61, 87, 38, 59, 33,  5, 11,  4, 58,
        59, 32,  5, 11,  4, 58, 59, 50, 27, 70, 38, 59, 32, 87, 35,  4, 38, 86,
        59, 56,  9,  5, 38, 86, 59, 88, 

In [0]:
def random_training_set(chunk_len=1000):    
    chunk = random_chunk(chunk_len)
    inp = text_to_tensor(chunk[:-1])   # omit the last token
    target = text_to_tensor(chunk[1:]) # omit the first token
    return inp, target

In [0]:
class Cartmanboi(nn.Module):
    def __init__(self, vocab_size, hidden_size, n_layers=1):
        super(Cartmanboi, self).__init__()
        # RNN attributes
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        # identiy matrix for generating one-hot vectors
        self.ident = torch.eye(vocab_size)
        # recurrent neural network
        self.rnn = nn.GRU(vocab_size, hidden_size, n_layers, batch_first=True)
        # a fully-connect layer that decodes the RNN output to
        # a distribution over the vocabulary
        self.decoder = nn.Linear(hidden_size, vocab_size)
    
    def forward(self, inp, hidden):
        # reshape the input tensor to [1, seq_length]
        inp = inp.view(1, -1)
        # generate one-hot vectors from token indices
        inp = self.ident[inp]
        # obtain the next output and hidden state
        output, hidden = self.rnn(inp, hidden)
        output = self.decoder(output.squeeze(0))
        return output, hidden

    def init_hidden(self):
        return torch.zeros(self.n_layers, 1, self.hidden_size)

In [0]:
def get_model_name(name, learning_rate, iteration):
    """ Generate a name for the model consisting of all the hyperparameter values

    Args:
        config: Configuration object containing the hyperparameters
    Returns:
        path: A string with the hyperparameter name and value concatenated
    """
    path = "model_{0}_lr{1}_iter{2}".format(name,
                                                   learning_rate,
                                                   iteration)
    return path

In [0]:
def evaluate(model, prime_str='I', predict_len=300, temperature=0.8):
    hidden = model.init_hidden()
    prime_input = text_to_tensor(prime_str)
    predicted = prime_str
    
    # Use priming string to "build up" hidden state
    for p in range(len(prime_str) - 1):
        _, hidden = model(prime_input[p], hidden)
    inp = prime_input[-1]
    
    for p in range(predict_len):
        output, hidden = model(inp, hidden)
        
        # Sample from the network as a multinomial distribution
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = int(torch.multinomial(output_dist, 1)[0])
        # Add predicted character to string and use as next input
        predicted_char = vocab_itos[top_i]
        predicted += predicted_char
        inp = text_to_tensor(predicted_char)

    return predicted

def train(model, num_iters=2000, lr=0.004):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    for it in range(num_iters):
        # get training set
        inp, target = random_training_set()
        # cleanup
        optimizer.zero_grad()
        # forward pass
        hidden = model.init_hidden()
        output, _ = model(inp, hidden)
        loss = criterion(output, target)
        # backward pass
        loss.backward()
        optimizer.step()

        if it % 100 == 99:
          print("[Iter %d] Loss %f" % (it+1, float(loss)))
          print("    " + evaluate(model, ' ', 300))
          model_path = get_model_name(model.name, lr, it+1)
          torch.save(model.state_dict(), model_path)


In [0]:
model = Cartmanboi(len(vocab),256,2)
train(model, num_iters=2000,lr=0.004)

[Iter 200] Loss 2.194304
     hiy an gols suy bige you wan the tout I's redid, ut inos its, tou'd is gou dout're tise node to  Gatte tou on dian is tave merte he ke mome just be beon'w attid ing! 
Man:	Oh way! 
Kangn:	I't wis, no gorarim wat, in the,  at Lad you it weot ke:	Dow beed le sap sit stor met, wit's of whack mere angis
[Iter 400] Loss 1.888162
     the gorkn for dayd Emat to I nevince Cartmana:	Mare!  
Butter:	How. Elle, Chring tual and home huve to kilk on to bet far watte to go cuvene pallist me hive bever the nees on aver cantery for it's kith can to spreatigro junte'll rightive cally parteve thin't man rong ulay fell the beat certmallyof m
[Iter 600] Loss 2.039475
     I ciple the clamp and theres a wish werin's a nay snuther shad at and me they ale not mand an and youh and stever mund there monet regry how. 
Butters:	No, where the ging they bo mike aptries gotpbackey a like that not have the poenther durn stars five the sarmang suppial they're and fuckar fat to m
[Iter 8

In [0]:
train(model, num_iters=3000,lr=0.004)

[Iter 200] Loss 1.579565
     stucking all be comice that me! 
Stan:	Well you love you could play on there bragett who handy me feel call back to thif ling muse good better mean! 
Dr. Terrince:	Futter missive a cal doing cholatus. 
Terrancon:	Oh mying you will a boytel. 
Cartman:	Does that my one starrit? me to lest on the dils 
[Iter 400] Loss 1.905129
     pyoppore!åÊ 
Guy:	What from the right of us ack. I don if you sume years of you.  
Butters:	Cartman it.  
Randy:	Whooo about a deel Chaigh it.  A sweet years? 
Kyle:	Uh, what can she doing everyone for your Dindror Randy life it? 
Cartman:	But you can free think and I supet a big and like the sure, 
[Iter 600] Loss 1.893726
     for the Indignaffay right well, but why deset! 
Cartman:	I got the needer for answer muney!  He's cots cool, I ass me. 
Cartman:	Oh my God!åÊåÊIt is of comnamen at me to thinkien in the frog ser who's like of my fise's who was out of tran. 
Cartman:	You got sure Kyle lame. A will master and ever onl
[Iter 8

In [0]:
a=evaluate(model, prime_str='Cartman:', predict_len=300, temperature=0.8)
print(a)

Cartman:	Yeah! You prodelam's parents ore was can say too about the streets. 
Butters:	Help gentlemen! Greets!  Oh, Mar, you guy? 
Dr Exeger Hindsod Bom frog feels just aren't Momacher!  
Cartman:	Help me a job.  
Cartman:	Dam, you really fuck a courthous to me, can't be week so we go watch me, real here! 



In [0]:
train(model, num_iters=5000,lr=0.004)

[Iter 200] Loss 1.325661
     like to your frick man... that we weld not anybody think. 
Scouts:	Yeah, too.  
Cartman:	Bear Kenny of you know what the time play stupid unterman with the XBody Damning human to the time tine my should be gonna stupid us. 
Cartman:	That's it of son. 
Jesus:	Maybe it's a fat. 
Leave God:	We, uh, dud
[Iter 400] Loss 1.087876
     the batts we can do that us farmach! Now can try it. 
Representy:	Look trouther Broflovski, sick and just down to me.  
Cartman:	I agn already be money! That me away Aiden. We just just true, who's there! 
Cartman:	Just startly sent time! 
Cartman:	Oh, you guys! The Fight is tired to get to at the g
[Iter 600] Loss 1.357392
     idea. 
Cartman:	Well, there's compleantly, Fills. You're those dollars. 
Stan:	Plan, but I don't know what it's a many some. Somebody are gonna have to stant a kays it's a chickler is gonna get me?! 
Cartman:	Szaul. 
Butters:	Have it was this lister gay with I againit to an and Phillip is can feel b
[Iter 8

In [0]:
a=evaluate(model, prime_str='Kyle:', predict_len=300, temperature=0.8)
print(a)

Kyle:	Well, I know, Wendy is the clud always sent to be a mountain around to go spend next to anything last that don't make a soon of along for that so much.  
Member 2:	Well you guys care on your uncleta I'm not a boy is cool. The way turn for a whole how doesn't happer out.  
Cartman:	What are you doin


In [2]:
### trying to tune it
class Cartmanboi1(nn.Module):
    def __init__(self, vocab_size, hidden_size, n_layers=1,dropout=0.2):
        super(Cartmanboi1, self).__init__()
        # RNN attributes
        self.name = "Cartmanboi1"
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        # identiy matrix for generating one-hot vectors
        self.ident = torch.eye(vocab_size)
        # recurrent neural network
        self.rnn = nn.GRU(vocab_size, hidden_size, n_layers, batch_first=True,dropout=dropout)
        # a fully-connect layer that decodes the RNN output to
        # a distribution over the vocabulary
        self.decoder = nn.Linear(hidden_size, vocab_size)
    
    def forward(self, inp, hidden):
        # reshape the input tensor to [1, seq_length]
        inp = inp.view(1, -1)
        # generate one-hot vectors from token indices
        inp = self.ident[inp]
        # obtain the next output and hidden state
        output, hidden = self.rnn(inp, hidden)
        output = self.decoder(output.squeeze(0))
        return output, hidden

    def init_hidden(self):
        return torch.zeros(self.n_layers, 1, self.hidden_size)


NameError: ignored

In [16]:
model_d = Cartmanboi1(len(vocab),256,2)
train(model_d, num_iters=2000,lr=0.002)

[Iter 200] Loss 2.306420
     les pueng on cous heke. 
Man'm And. Here and on thing theree the reante Manty:	I the ghaber, herg ce at all we cone se outel anle you the Mrefat houpstering theonn's wut monl. 
Carese:	AAsh't the att prat, tepe ound oul ont! 
Kylle:	Lall to dowing. 
Leatting tor ke leas? But mank and.  
uttersss:	Le
[Iter 400] Loss 1.950507
     maclild steed to and wan to somato as my No me Pack a marena home waine peoffian it, I'm a the parted we wars frople is of seaple and you very now andevery warsica sook we wers is have one wan to realle store ane on prillle Savery ne is tave te Maghe and I tame it onace this get frohing the see have
[Iter 600] Loss 1.679303
     Frover gots in then just everybody. 
To. Garrigot:	Mr. DackEy, Goy, butare to got a thit dond to tele. 
Park:	Sure with, head, beat the olp hould sure your me planerith thole way sin we donith a doing to me senit at sould bele...  
Olrower:	Pus sucem, what it's all me for the resivert, corecE you le
[Iter 8

In [38]:
model_d1 = Cartmanboi1(len(vocab),256,2,dropout=0.3)
train(model_d1, num_iters=5000,lr=0.002)

[Iter 100] Loss 2.589861
     annivery Hy unwan:	Io thaas ibe sane youn suve at hat ron:	ill oor souve dly Prikor eh ut le 

Caadhy foon  
Cannenin:	Yhema ke le! I fot won ow.??? 
Sran:	Sparsriventhy the dee the torl hity et Fy yous ap hade been:	Yy 1in:	I dol  
ae! Irllw you fpikve the op ourt all got! 	fnon:	Wease arethan:	Kla
[Iter 200] Loss 2.166937
     Jerders teaed do thounttunkid sobme wine! 
Carm1:	And! I doous notee here anderd it an we ing to relle, he mir, Me tforew on ike git ang thone sand Weint that maod be sousw the 
Pane:	Dover, leere thesp tor is fus, Ceryout, I king got, to ne? po you. Stoor and! 
Ms Clacbe:	YeOh! 
Ranq:Cale. 
Ricere:
[Iter 300] Loss 2.166804
     Glay wannatore of one thet gonna an of in-alpar. 
Mremyme:	Meldone ge. 
Mamterbe:	No! 
Mandy:	Yow, I kelig, Phat there wat of the has ghe you carl Buth! Ste pobter e ver the tuit? 
Taacy:	Whathere, you he me. 
Are allice:	Erousser hast beaso MrPackery! Dide sey rouss the sone anyones we! 
Cartmers:	
[Iter 4

KeyboardInterrupt: ignored

In [39]:
model_d2 = Cartmanboi1(len(vocab),256,2,dropout=0.2)
train(model_d2, num_iters=5000,lr=0.002)

[Iter 100] Loss 2.424732
     /Or hacme:	Wout on mod Gofreile ton dou hoLer. 
Sarghon! 
Bandy Cos:	Ste:	Delost I cou thot shoan:	Mry. 
Yyourt se hove I topet rert ol cfit. 
Merl aple, son:	I dist ive  his:	Wu to spanl:	Aud! 
Nyan. Ther and fo tou mele ande me, toum f we rermet your witm Irea thas out if't ther mos leere hill und
[Iter 200] Loss 2.123166
     coreando pickmadlers I'm you! 
CantelM:	At antter Crame handen taeple couk e teriepre! Not ouriend wied ow olle! 
Cartman:	UOh Someke dond Mily onere.  
Cartman:	Hecp? 
Cartmal:	We ant is poon't rederine boold of youk aik Sterplend! OOAleen. 
CErtman:	Sald your woke? 
Liegere:	Will sharka deaplo fob
[Iter 300] Loss 1.826243
     all you han of on wers just the goed the anton don't asend wing the fin the dowting the can the with the welp that shesellernt, you keve were the con anture hay conded the the en, the an the in this thoucg the cwaclle't selly outt a cethorbest, whe carous in the witming not in time to some Cantmisth
[Iter 4

In [55]:
net = Cartmanboi1(len(vocab),256,2)
model_path = get_model_name(net.name,
learning_rate=0.002, iteration=4800)
state = torch.load(model_path)
net.load_state_dict(state)
p = evaluate(net, prime_str='Kyle:', predict_len=300, temperature=0.8)
print(p)

Kyle:	I don't need ever home. 
Kyle:	You don't see it to believe it before... damn you! 
Stan:	All right, just leads to kill you! 
Mr. Triscotti:	Tell it all the Holanch of a towel 
Chresi:	You all know that ban Ration problem. 
Stan:	I love a buster. I've got him to get a danating used to pull it... oka


In [0]:
train(model_d2, num_iters=5000,lr=0.002)

[Iter 100] Loss 0.983626
     okay. 
Butters:	Well fine this teacher, you still get our sedting? Don't you see? No, Mr. Marsh. I want that sonofabilation waiting my better to stay that the damage of Christean.  
Time Child:	No, no- I just show the holorinal on our side.  
Stan:	Yeah, it sure what I do for that sure? 
Kyle:	But f
[Iter 200] Loss 1.045090
     has got it! 
Randy:	No, nobody seem as it getting lett Hitler simp. - ROS RRRRRRRRRDUR? Why was comin' in the way to the point? I want to! 
Gary:	This is a memonity to be butt. 
Momen:	They was all thing was a suts... Steven. 
Cesar:	Helpo the county for be having sex with a whole life. 
Cartman:	No
[Iter 300] Loss 1.040637
     here! 
Butters:	For making your family? 
Kyle:	Ahhh? 
Terrance:	Listens, I really don't see no big neare the hell. There's no stucing plan, students. A pretty damn! 
Driver:	MeAl Oprihata! 
Otter Leader:	Hi, nobody else in the fuck, Mr. Spenduding confused in the street on "per"1:	Oh yeah!  
Cartman
[Iter 4